# Coder Hussam Qassim

# Build 3D Dataset

In [4]:
import numpy as np

np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

# PCA

In [ ]:
'''
Principal Component Analysis (PCA) is by far the most popular dimensionality reduction algorithm.
First it identifies the hyperplane that lies closest to the data, and then it projects the data onto it.
'''
'''
Scikit-Learn’s PCA class implements PCA using SVD decomposition just like we did before. The following code 
applies PCA to reduce the dimensionality of the dataset down to two dimensions (note that it automatically 
takes care of centering the data)
'''
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
X2D = pca.fit_transform(X)

'''
After fitting the PCA transformer to the dataset, you can access the principal components using the components_
variable (note that it contains the PCs as horizontal vectors, so, for example, the first principal component 
is equal to pca.components_.T[:, 0] )
'''

### Explained Variance Ratio

In [1]:
'''
Another very useful piece of information is the explained variance ratio of each principal component, available
via the explained_variance_ratio_ variable. It indicates the proportion of the dataset’s variance that lies 
along the axis of each principal component
'''
print(pca.explained_variance_ratio_)

'''
This tells you that 84.2% of the dataset’s variance lies along the first axis, and 14.6% lies along the second
axis. This leaves less than 1.2% for the third axis, so it is reasonable to assume that it probably carries 
little information
'''

NameError: name 'pca' is not defined

### Choosing the Right Number of Dimensions

In [2]:
'''
Instead of arbitrarily choosing the number of dimensions to reduce down to, it is generally preferable to
choose the number of dimensions that add up to a sufficiently large portion of the variance (e.g., 95%).
Unless, of course, you are reducing dimensionality for data visualization — in that case you will generally 
want to reduce the dimensionality down to 2 or 3. The following code computes PCA without reducing 
dimensionality, then computes the minimum number of dimensions required to preserve 95% of the training set’s 
variance
'''
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

'''
You could then set n_components=d and run PCA again. However, there is a much better option: instead of 
specifying the number of principal components you want to preserve, you can set n_components to be a float 
between 0.0 and 1.0 , indicating the ratio of variance you wish to preserve
'''
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)

'''
Yet another option is to plot the explained variance as a function of the number of dimensions (simply plot
cumsum). There will usually be an elbow in the curve, where the explained variance stops growing fast. You can
think of this as the intrinsic dimensionality of the dataset. In this case, you can see that reducing the 
dimensionality down to about 100 dimensions wouldn’t lose too much explained variance
'''

NameError: name 'PCA' is not defined

### PCA for Compression

In [ ]:
'''
Obviously after dimensionality reduction, the training set takes up much less space. For example, try applying 
PCA to the MNIST dataset while preserving 95% of its variance. You should find that each instance will have 
just over 150 features, instead of the original 784 features. So while most of the variance is preserved, 
the dataset is now less than 20% of its original size! This is a reasonable compression ratio, and you can see 
how this can speed up a classification algorithm (such as an SVM classifier) tremendously. It is also possible 
to decompress the reduced dataset back to 784 dimensions by applying the inverse transformation of the PCA 
projection. Of course this won’t give you back the original data, since the projection lost a bit of 
information (within the 5% variance that was dropped), but it will likely be quite close to the original data. 
The mean squared distance between the original data and the reconstructed data (compressed and then 
decompressed) is called the reconstruction error. For example, the following code compresses the MNIST dataset
down to 154 dimensions, then uses the inverse_transform() method to decompress it back to 784 dimensions. 
'''
pca = PCA(n_components = 154)
X_mnist_reduced = pca.fit_transform(X_mnist)
X_mnist_recovered = pca.inverse_transform(X_mnist_reduced)

### Incremental PCA

In [ ]:
'''
One problem with the preceding implementation of PCA is that it requires the whole training set to fit in
memory in order for the SVD algorithm to run. Fortunately, Incremental PCA (IPCA) algorithms have
been developed: you can split the training set into mini-batches and feed an IPCA algorithm one	mini-
batch at a time. This is useful for large training sets, and also to apply PCA online (i.e., on the fly, as new
instances arrive). The following code splits the MNIST dataset into 100 mini-batches (using NumPy’s 
array_split() function) and feeds them to Scikit-Learn’s IncrementalPCA class 5 to reduce the dimensionality 
of the MNIST dataset down to 154 dimensions (just like before). Note that you must call the partial_fit()
method with each mini-batch rather than the fit() method with the whole training set
'''
from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_mnist, n_batches):
    inc_pca.partial_fit(X_batch)
X_mnist_reduced = inc_pca.transform(X_mnist)

'''
Alternatively, you can use NumPy’s memmap class, which allows you to manipulate a large array stored in a 
binary file on disk as if it were entirely in memory; the class loads only the data it needs in memory, when
it needs it. Since the IncrementalPCA class uses only a small part of the array at any given time, the memory
usage remains under control. This makes it possible to call the usual fit() method, as you can see in the 
following code
'''
X_mm = np.memmap(filename, dtype="float32", mode="readonly", shape=(m, n))
batch_size = m // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)

### Randomized PCA

In [ ]:
'''
Scikit-Learn offers yet another option to perform PCA, called Randomized PCA. This is a stochastic algorithm
that quickly finds an approximation of the first d principal components. Its computational complexity is
O(m × d2 ) + O(d3 ), instead of O(m × n2) + O(n3), so it is dramatically faster than the previous algorithms
when d is much smaller than n
'''
rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_reduced = rnd_pca.fit_transform(X_mnist)

### Kernel PCA

In [ ]:
'''
We discussed the kernel trick, a mathematical technique that implicitly maps instances into a very 
high-dimensional space (called the feature space), enabling nonlinear classification and regression with 
Support Vector Machines. Recall that a linear decision boundary in the high-dimensional feature space 
corresponds to a complex nonlinear decision boundary in the original space. It turns out that the same trick
can be applied to PCA, making it possible to perform complex nonlinear projections for dimensionality reduction.	This	is	called	Kernel	PCA	(kPCA). 6 	It	is	often	good	at
preserving clusters of instances after projection, or sometimes even unrolling datasets that lie close to a 
twisted manifold. For example, the following code uses Scikit-Learn’s KernelPCA class to perform kPCA with 
an RB kernel
'''
from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

#### Selecting a Kernel and Tuning Hyperparameters

In [ ]:
'''
As	kPCA is an unsupervised learning algorithm, there is no obvious performance measure to help you select the best
kernel and hyperparameter values. However, dimensionality reduction is often a preparation step for a 
supervised learning task (e.g., classification), so you can simply use grid search to select the kernel and 
hyperparameters that lead to the best performance on that task. For example, the following code creates a 
two-step pipeline, first reducing dimensionality to two dimensions using kPCA, then applying Logistic 
Regression for classification. Then it uses GridSearchCV to find the best kernel and gamma value for kPCA in
order to get the best classification accuracy at the end of the pipeline
'''
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = Pipeline([
            ("kpca", KernelPCA(n_components=2)),
            ("log_reg",	LogisticRegression())
        ])
param_grid = [{
            "kpca__gamma":	np.linspace(0.03,	0.05,	10),
            "kpca__kernel":	["rbf",	"sigmoid"]
        }]
grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X, y)

# The best kernel and hyperparameters are then available through the best_params_ variable
print(grid_search.best_params_)

'''
Another approach, this time entirely unsupervised, is to select the kernel and hyperparameters that yield
the lowest reconstruction error. However, reconstruction is not as easy as with linear PCA. Here’s why. Thanks
to the kernel trick, this is mathematically equivalent to mapping the training set to an infinite-dimensional
feature space using the feature map φ, then projecting the transformed training set down to 2D using linear PCA.
Notice that if we could invert the linear PCA step for a given instance in the reduced space, the reconstructed
point would lie in feature space, not in the original space (e.g., like the one represented by an x in the 
diagram). Since the feature space is infinite-dimensional, we cannot compute the reconstructed point, and 
therefore we cannot compute the true reconstruction error. Fortunately, it is possible to find a point in the
original space that would map close to the reconstructed point. This is called the reconstruction pre-image.
Once you have this pre-image, you can measure its squared distance to the original instance. You can then
select the kernel and hyperparameters that minimize this reconstruction pre-image error. You may be wondering
how to perform this reconstruction. One solution is to train a supervised regression model, with the projected
instances as the training set and the original instances as the targets. Scikit-Learn will do this 
automatically if you set fit_inverse_transform=True , as shown in the following code
'''
rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.0433,
                    fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)

'''
NOTE: By default, fit_inverse_transform=False and KernelPCA has no inverse_transform() method. This method 
only gets created when you set fit_inverse_transform=True.
'''
# You can then compute the reconstruction pre-image error
from sklearn.metrics import mean_squared_error

mean_squared_error(X, X_preimage)

'''
Now you can use grid search with cross-validation to find the kernel and hyperparameters that minimize this
pre-image reconstruction error
'''

### LLE

In [ ]:
'''
Locally Linear Embedding (LLE) is another very powerful nonlinear dimensionality reduction (NLDR) technique.
It is a Manifold Learning technique that does not rely on projections like the previous algorithms. In a 
nutshell, LLE works by first measuring how each training instance linearly relates to its closest neighbors 
(c.n.), and then looking for a low-dimensional representation of the training set where these local 
relationships are best preserved (more details shortly). This makes it particularly good at unrolling twisted
manifolds, especially when there is not too much noise. For example, the following code uses Scikit-Learn’s 
LocallyLinearEmbedding class to unroll the Swiss roll. As you can see, the Swiss roll is completely unrolled
and the distances between instances are locally well preserved. However, distances are not preserved on a 
larger scale: the left part of the unrolled Swiss roll is squeezed, while the right part is stretched.
Nevertheless, LLE did a pretty good job at modeling the manifold.
'''
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced = lle.fit_transform(X)

# Other Dimensionality Reduction Techniques

In [ ]:
'''
There are many other dimensionality reduction techniques, several of which are available in Scikit-Learn.
Here are some of the most popular:

- Multidimensional Scaling (MDS) reduces dimensionality while trying to preserve the distance between the 
instances. 

- Isomap creates a graph by connecting each instance to its nearest neighbors, then reduces dimensionality 
while trying to preserve the geodesic distances 9 between the instances. 

- t-Distributed Stochastic Neighbor Embedding (t-SNE) reduces dimensionality while trying to keep similar 
instances close and dissimilar instances apart. It is mostly used for visualization, in particular to 
visualize clusters of instances in high-dimensional space (e.g., to visualize the MNIST images in 2D).

- Linear Discriminant Analysis (LDA) is actually a classification algorithm, but during training it learns
the most discriminative axes between the classes, and these axes can then be used to define a hyperplane
onto which to project the data. The benefit is that the projection will keep classes as far apart as possible,
so LDA is a good technique to reduce dimensionality before running another classification algorithm such as 
an SVM classifier.
'''